# Домашнее задание 2

В этом задании вам предстоит реализовать контрастивное обучние эмбеддера, посмотреть на его влияние на задаче классификации и отбора кандидатов.

Языковое моделирование рассматривать не будем в силу дороговизны подхода.

## Часть 1. Triplet loss на стероидах

Вам поставили задачу: на фиксированном множестве точек произвести классификацию, при этом множество таково, что качество на исходных данных неприемлемо. Что делать? Последуем совету из лекции и реализуем контрастивное обучение.

В этом и последующем задании вам предстоит реализовать предобучение некоторого простого эмбеддера на домен.

Эмбеддинги заморожены -- будем дообучать только полносвязную голову.

In [ ]:
from dataclasses import dataclass

from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split

import torch
from torch import nn
import torch.nn.functional as F
import numpy as np

In [ ]:
SEED = 42

@dataclass
class DatasetConfig:
    n_samples: int = 5000
    n_features: int = 32
    n_classes: int = 8
    n_clusters_per_class: int = 2
    n_informative: int = 5
    random_state: int = SEED

@dataclass
class SplitConfig:
    random_state: int = SEED
    test_size: float = 0.25

In [ ]:
X, y = make_classification(**DatasetConfig().__dict__)

X_train, X_test, y_train, y_test = train_test_split(X, y, **SplitConfig().__dict__)
X_train, y_train = torch.from_numpy(X_train).float(), torch.from_numpy(y_train).float()

## 1.Визуализация данных - 1 баллов

Напишите функцию `plot_tsne(data, labels, **kwargs)`, принимающую на вход матрицу эмбеддингов и метки классификации и строящую t-SNE-разложение на плоскости. Изобразите его, раскрасив классы по цветам. Зафиксируйте `random_state` при построении.

In [ ]:
import pandas as pd
from sklearn.manifold import TSNE
import matplotlib.pyplot as plt
import seaborn as sns


# ---- Ваш код здесь ----
print("Красиво накидываем точки на плоскость")
# ---- Конец кода ----

## 2.Базовый классификатор - 1 балл

Выберите алгоритм классификации, который вам больше нравится (но советую взять kNN). Возьмите accuracy в качестве метрики качества классификации. Оцените базовое качество на тестовых данных.

In [ ]:
from sklearn.neighbors import KNeighborsClassifier


# ---- Ваш код здесь ----
print("Плотно классифицируем тест")
# ---- Конец кода ----

## 3.TripletLoss - 5 баллов

Заметим, что в случае L2-нормилизованных векторов:

$$\max\left(0,\|f(x)-f(x^+)\|^2_2-\|f(x)-f(x^-)\|^2_2+\varepsilon\right)=\max\left(0,f(x)f^T(x^-)-f(x)f^T(x^+)+\varepsilon\right)$$

Выше записан triplet loss. Его мы будем реализовывать, но с некоторой упрощающей модификацией.

Пусть $D=\{x_i, y_i\}_i$ -- выборка классификации. Пусть $S=XX^T$. Кто будет формировать позитивы? Такие $j\neq i:\;y_i=y_j$ -- диагональ $S$ вырезается. При этом по матрице $S$ можно сформировать два непересекающихся множества: позитивов $P$ и негативов $N$ (как оставшихся пар, где $y_i\neq y_j$). Пусть $L$ -- минимальная мощность этих двух множеств. Возьмем $\hat{P}=\{p_i\}_i$, $\hat{N}=\{n_i\}_i$ как сэмплы размера $L$ из $P$ и $N$ соответственно. Тогда итоговая функция ошибки выглядит так:

$$\mathcal{L}=\frac{1}{L}\sum\limits_{i=1}^L\max\left(0, n_i-p_i+\varepsilon\right)$$

В чем модификация? В том, что в паре позитивов и негативов не обязательно должен быть один и тот же якорный элемент. И это работает.

Реализуйте callable-класс `TripletLoss` по описанию.

In [ ]:
# ---- Ваш код здесь ----
class TripletLoss():
    
    def __init__(self, margin, random_state=None):
        self.margin = margin
        self.random_state = random_state

    def __call__(self, x, labels):
        raise NotImplementedError()
# ---- Конец кода ----


In [ ]:
criterion = TripletLoss(0.2, random_state=101)


objects = torch.tensor(
    [[-1.7651, -1.5979,  0.1042,  0.3825, -0.9419, -0.2580, -0.6087, -0.1711,
        1.3922,  0.8548, -0.9251,  0.6989,  0.4238, -0.1330,  0.2985],
    [ 1.6144,  0.0627,  0.3424, -0.8591,  0.1869, -0.8598, -0.7200,  0.9449,
        -0.1684,  1.0282, -1.2377, -1.2640,  0.7469,  1.9605, -0.1214],
    [ 1.1143, -0.6948,  0.3739, -1.1461,  0.6456, -0.3360, -0.8111, -0.8861,
        0.7176, -0.6235, -0.9364,  0.6174,  2.7212, -2.0703, -2.2571],
    [ 0.7525,  2.1028,  2.7782,  0.5040, -1.5791,  1.5342,  0.0816,  0.3245,
        -0.0857, -0.5992, -1.4339,  0.0897, -1.5096,  0.1428, -0.1488],
    [-0.7518,  0.2623, -0.4958, -1.6063,  0.2537, -0.1137,  0.3985,  1.0155,
        0.1874, -0.4300, -1.2309,  1.5760, -1.3176,  1.5355,  1.8471],
    [ 1.9290, -0.3236,  0.4303,  0.7111,  1.4234,  1.7901,  0.2216, -1.5471,
        0.9389, -0.3012, -1.6487,  1.5765, -1.1450,  0.3260,  0.4909],
    [ 0.7837, -0.8004, -0.0929, -1.2220,  2.2333,  0.3288, -0.5222, -0.7202,
        0.6147,  1.8012, -0.2388, -0.2539,  0.0191, -0.0104,  0.5717],
    [-0.2709, -1.7985, -0.3959, -1.1190,  0.8644,  0.3008, -1.0336, -0.1251,
        -0.3357,  0.7938,  3.2090, -0.4332, -0.0496, -0.2672,  0.9690],
    [-0.1109,  0.4130,  0.7406, -1.2446, -0.4252,  2.5128, -0.2765,  0.6845,
        1.1965,  1.4173, -1.4604,  0.2515,  0.6387, -1.8519,  1.1899],
    [-0.1781, -0.7473, -0.1015,  0.2280, -1.5815,  0.1535, -1.3912, -2.2026,
        1.0496,  0.3547,  0.8897, -0.6482,  0.0133,  1.0124, -0.4452]])

labels = torch.LongTensor([1, 1, 2, 2, 3, 3, 4, 4, 5, 5])
assert abs(criterion(objects, labels).item() - 0.29527) < 1e-4

## 4.Модель и функция обучения - 8 балла

Реализуйте класс `MLP`, полносвязную нейронную сеть. Выбирайте на свой вкус.

Реализуйте функцию `domain_adaptation`, стандартный цикл batch-обучения модели. Батч можно сэмплировать произвольно через `choice`.

Требуется выбить на тесте 0.60 точности.

In [ ]:
# ---- Ваш код здесь ----
print("Задаем полносвязную простенькую сетку")
# ---- Конец кода ----

In [ ]:
# ---- Ваш код здесь ----
print("Учим эмбеддер")
# ---- Конец кода ----

## 5.Итоговое качество - 1 балл

Отобразите новое распределение t-SNE-координат и посчитайте тестовую метрику. Сделайте краткий вывод.

In [ ]:
# ---- Ваш код здесь ----
print("Визуализируем успех")
# ---- Конец кода ----

In [ ]:
# ---- Ваш код здесь ----
print("Считаем точность на тесте")
# ---- Конец кода ----

## Часть 2. Triplet loss на чем-то посерьезнее

Рассмотрим теперь более живую задачу классификации. Будем работать с новостными группами.

Постановка та же, только возьмем теперь предобученный эмбеддер с HF. Эмбеддинги заморожены -- будем дообучать только полносвязную голову.

In [ ]:
from sklearn.datasets import fetch_20newsgroups
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report

In [ ]:
categories = [
    "sci.space",
    "sci.med",
    "sci.electronics",
    "comp.os.ms-windows.misc",
    "comp.sys.ibm.pc.hardware",
    "comp.sys.mac.hardware"
]

newsgroups_train = fetch_20newsgroups(subset="train", categories=categories)
newsgroups_test = fetch_20newsgroups(subset="test", categories=categories)

X_train = newsgroups_train.data
X_test = newsgroups_test.data

y_train = newsgroups_train.target
y_test = newsgroups_test.target

In [ ]:
def test_logreg(X_train_mapped, y_train, X_test_mapped, y_test, target_names=newsgroups_test.target_names):
    clf = LogisticRegression(max_iter=10000)
    clf.fit(X_train_mapped, y_train)

    y_pred = clf.predict(X_test_mapped)
    score = accuracy_score(y_test, y_pred)
    report = classification_report(y_test, y_pred, target_names=target_names)

    print(f"Accuracy: {score:.3f}")
    print(f"Classification Report: {report}")

## 1.Выбираем англоязычный эмбеддер и формируем эмбеддинги - 1 балл

Выберите небольшой англоязычный sentence-трансформер, обученный на семантику, и прогоните через нее тексты обучения и тестирования.

Замерьте базовое качество классификатора на этих эмбеддингах.

Опционально: крайне рекомендую прогнать `plot_tsne` из прошлой части на тесте.

In [ ]:
import torch
from tqdm import tqdm
from sentence_transformers import SentenceTransformer

# ---- Ваш код здесь ----
print("Определяем модель, получаем эмбеддинги, визуализируем тест через t-SNE")
# ---- Конец кода ----

## 2.Обучаем эмбеддинги под задачу - 1 балл

Теперь точно придется обратиться к Части 1. Необходимо взять `domain adaptation` и обучить эмбеддиги на домен.

In [ ]:
# ---- Ваш код здесь ----
print("Доменно адаптируемся")
# ---- Конец кода ----

## 3.Замеряем качество - 1 балл

Обучитите базовый классификатор на новом пространстве эмбеддингов, сравните результаты, напишите вывод.

Опционально: вновь крайне рекомендую прогнать `plot_tsne` из прошлой части на тесте, только уже в новом пространстве.

In [ ]:
# ---- Ваш код здесь ----
print("Радуемся росту качества")
# ---- Конец кода ----

# Часть 3. Контрастивное обучение для поискового отбора кандидатов.

Эта часть будет более кейс-ориентированной, мы разберем сценарий, в котором контрастивное обучение является стандартной практикой улучшения качества модели на конечной задаче.

Бизнес-кейс:
> Требуется улучшить этап отбора кандидатов в поисковой веб-системе. На текущий момент в качестве кандгена (кандидатогенерации) используется BM25 и обратный индекс. BM25 уже тюнили, дальше качество нарастить не выходит. В качестве бизнес-метрики можем взять производные поведенческого отклика, например, CTR@K или timespent на выдаче и документах.

Очевидным направлением развития является построение нейросетевого кандгена. Обычно в описанных случаях действуют следующим образом:
0. Выбирают ML-метрику, которую хотелось бы оптимизировать. Для кандгена катастрофически важно выдать как можно больше релеватных документов в пределах фиксированной длины выдачи, поэтому подходящая метрика -- Recall@K. Мы будем использовать ее модификацию, но об этом позже.
1. Сэмплируют запросы из потока / формируют специфичные корзины запросов в зависимости от дополнительных бизнес-требований. Давайте считать, что они отсутствуют. Тут обязателен контроль их качества, можно исходить из символьных эвристик или применять LLM для классификации, как вы это делали в предыдущей домашке.
2. Обкачивают поисковый движок, формируя глубокие выдачи. Эпитет "глубокие" относится к глубине погружения пользователя в выдачу, то есть предельные позиции взаимодействия с документами. Так вот для обучения требуется брать документов в избытке, в том числе те, с которыми пользователь никогда бы не повзаимодействовал. В целом, длина выдачи 1000 -- отличный выбор. Предварительно есть смысл сгладить все условия отбора по BM25.
3. Разметка пар запрос-документ на задачу релевантности. LLM -- вновь отличный выбор. Разметка порядковая, но может быть как бинарной, так и n-арной. Важно сформировать определение "релевантного" документа, то есть определить порог, по которому мы будем считать документ подходящим под запрос.
4. Релевантные пары запрос-документ берем в качестве позитивов, выбираем базовый эмбеддер и учим его контрастивно как bi-энкодер на эту выборку, негативы можем формировать в режиме in-batch.
5. Если все сделано верно (данных достаточно, гиперпараметры подобраны, код не багованный), естественным следствием будет рост качества, поздравляю.

Датасет, на котором мы будем строить кандген -- MS Marco Dev. В качестве эмбеддера вы вольны использовать любые модели, которые не учились на MS Marco, например, `"microsoft/deberta-v3-small"`.

Мы привыкли в основном, что датасеты собраны на HF, но в этот раз рассмотрим другую библиотеку для работы с датасетами, `ir_datasets` ([API](https://ir-datasets.com/python.html)). "IR" от Information Retrieval - библиотека содержит инструменты работы с датасетами поиска. Также в коде будет использоваться `polars` ([API](https://docs.pola.rs/api/python/stable/reference/index.html)), аналог всеми известной `pandas`, только на порядки быстрее.

Описание датасета читайте [тут](https://ir-datasets.com/msmarco-passage.html#msmarco-passage/dev/judged).

In [ ]:
import torch
from torch import nn
from torch.nn import functional as F
from torch.utils.data import Dataset, DataLoader
from torch.optim import AdamW
from torch.optim.lr_scheduler import OneCycleLR

from transformers import AutoModel, AutoTokenizer

import re
from dataclasses import dataclass
from collections import defaultdict
from functools import partial
from tqdm import tqdm

import numpy as np
import ir_datasets
import polars as pl

import faiss
from sklearn.model_selection import train_test_split

In [ ]:
seed = 42
np.random.seed(seed)
torch.manual_seed(seed)

In [ ]:
@dataclass
class Columns:
    query_id: str = "query_id"
    doc_id: str = "doc_id"
    index_id: str = "index_id"
    text: str = "text"
    qrels_relevance: str = "relevance"


@dataclass
class DatasetConfig:
    sampled_index_size: int = 150_000
    relevance_threshold: int = 1
    test_size: float = 0.2

In [ ]:
dataset = ir_datasets.load("msmarco-passage/dev/judged")

columns = Columns()
dataset_config = DatasetConfig()

In [ ]:
queries = pl.DataFrame(dataset.queries_iter()).select(
    pl.col(columns.query_id).cast(pl.Int32),
    pl.col(columns.text)
)

qrels = pl.DataFrame(dataset.qrels_iter()).drop("iteration").select(
    pl.col(columns.query_id).cast(pl.Int32),
    pl.col(columns.doc_id).cast(pl.Int32),
    pl.col(columns.qrels_relevance).cast(pl.Int32)
)

documents = pl.DataFrame(dataset.docs_iter()).select(
    pl.col(columns.doc_id).cast(pl.Int32),
    pl.col(columns.text)
)

In [ ]:
target_document_ids = qrels[columns.doc_id].unique().to_list()
sampled_document_ids = np.random.default_rng().integers(dataset.docs_count(), size=dataset_config.sampled_index_size).tolist()

sampled_documents = documents.filter(pl.col(columns.doc_id).is_in(sampled_document_ids + target_document_ids)).with_row_index(columns.index_id)
len(target_document_ids), len(sampled_document_ids)

In [ ]:
train_qrels, test_qrels = train_test_split(qrels, test_size=dataset_config.test_size)

train_queries = queries.filter(pl.col(columns.query_id).is_in(train_qrels[columns.query_id].implode()))
test_queries = queries.filter(pl.col(columns.query_id).is_in(test_qrels[columns.query_id].implode()))

train_documents = sampled_documents.filter(pl.col(columns.doc_id).is_in(train_qrels[columns.doc_id].implode()))
test_documents = sampled_documents.filter(pl.col(columns.doc_id).is_in(test_qrels[columns.doc_id].implode()))


## 0.Дополните параметры конфигов в зависимости от вашей реализации модели и обучения

Задайте параметры токенизации, обучения, значения параметров функции ошибки и дополнительные параметры модели.
Что стоит добавить в конфиг модели:
- возможность заморозить хвост и дообучать только голову (делается через регулярки и `model.named_parameters()`)
- снижение размерности за счет дополнительной полносвязной сети
- параметры для этой дополнительной сети

Описание полей конфига функции ошибки читай ниже.

In [ ]:
@dataclass
class TrainTestConfig:
    device: str = torch.device(["cpu", "cuda"][torch.cuda.is_available()])
# ---- Ваш код здесь ----
# ---- Конец кода ----

@dataclass
class ModelConfig:
    model_name: str
# ---- Ваш код здесь ----
# ---- Конец кода ----

@dataclass
class LossConfig:
# ---- Ваш код здесь ----
    thrsh: float = ...
    temperature: float = ...
# ---- Конец кода ----

In [ ]:
columns = Columns()
dataset_config = DatasetConfig()
train_test_config = TrainTestConfig()


# ---- Ваш код здесь ----
model_config = ModelConfig(...)
# ---- Конец кода ----

## 1.Датасет - 4 баллов

Напишите класс `DenseRetrievalDataset`, наследованный от `Dataset`, который внутри формирует множество релевантных пар и выдает на каждый индекс произвольную пару оттуда вместе с `query_id` и `doc_id`.

Напишите также функцию `train_collate_fn` для DataLoader'а, которая внутри токенизирует батчем текст запроса и документа и отдает кортеж из тензоров, в которые включаются id запросов и документов, токены запросов и документов.

In [ ]:
class DenseRetrievalDataset(Dataset):
    def __init__(self, queries, documents, qrels, columns, config):
        self.columns = columns
        self.config = config

        self.queries = queries
        self.documents = documents
# ---- Ваш код здесь ----
        self.qrels = ...
# ---- Конец кода ----



In [ ]:
def train_collate_fn(data, tokenizer, config):
# ---- Ваш код здесь ----
    raise NotImplementedError()
# ---- Конец кода ----

## 2.Функция ошибки - 5 баллов

Реализуйте класс `ContrastiveLoss`, который реализует расчет следующей функции ошибки:
$$\mathcal{L}=\mathbb{E}_T\text{CrossEntropy}\left(q_iD^T-B_i, M_i\right)$$
$$T=\{Q, D\},\quad Q=\{q_i\big|q_i\in\mathbb{R}^n,\|q_i\|_2=1\}_{i=1}^N,\quad D=\{d_i\big|d_i\in\mathbb{R}^n,\|d_i\|_2=1\}_{i=1}^N$$
$$(q_i, d_i) \,-\,\text{позитивная пара}$$
$$M_i\in[0,1]^N,\quad \forall{j}\in\overline{1,N}:\;M[j]=\frac{[q_i = q_j]}{\sum\limits_k{[q_i=q_k]}}$$
$$B_i\in[0,1]^N,\quad \forall{j}\in\overline{1,N}:\;M[j]=b*[q_i = q_j]$$
$$b\,-\,\text{вещественный гиперпараметр}$$

Смысл $b$ смотрите в [статье LaBSE](https://arxiv.org/pdf/2007.01852), _Additive Margin Softmax_.

Фактически вы напишите InfoNCE с in-batch-негативами.

Подсказка: не упаковывайте расчет $M_i$ внутрь функции ошибки, сделайте ее внешней. Она вам пригодится в функции обучения.

In [ ]:
# ---- Ваш код здесь ----
class ContrastiveLoss(nn.Module):
    def __init__(self, thrsh, temperature):
        super().__init__()
        self.thrsh = thrsh
        self.temperature = temperature

    def forward(self, queries, documents, labels):
        raise NotImplementedError()
# ---- Конец кода ----

In [ ]:
criterion = ContrastiveLoss(0.1, 0.05)


queries = torch.tensor(
    [[ 0.5803,  0.9579, -1.7393,  0.8502,  1.0579,  1.1222, -1.3303,  2.1554,
        -0.2404,  1.7580,  0.1433,  0.6232, -0.9371,  0.7069,  0.9060],
    [ 1.4968, -0.4212, -0.3566, -0.1982,  0.3722,  0.4442,  1.0164,  0.8380,
        -0.5248, -1.1686,  1.3973, -0.6910, -0.5832, -0.2636, -1.0497],
    [ 0.1836, -1.2159, -0.5191, -1.5825,  0.4003, -0.6419, -1.1341,  0.2970,
        -1.1792,  2.1851,  2.3077,  0.3735,  1.4981,  0.6243,  1.2269],
    [-2.7559, -0.2543,  0.6742, -0.0188, -0.3204,  0.2138,  0.2517, -2.2059,
        -1.3797, -1.5980, -1.3527,  1.5497, -0.7449,  0.6207, -1.8088],
    [ 0.7241,  1.2993,  0.8433,  0.1442, -1.0798,  1.7103,  0.0768, -1.0067,
        -0.4282,  0.7578, -0.0629, -0.4202,  0.8126, -0.1174,  0.8947],
    [ 1.7049, -0.6559,  0.4521, -0.4866,  0.2823, -0.0065, -0.6142,  0.9237,
        -0.6907,  0.6034,  0.2700,  1.0026,  0.9323,  1.3452, -1.1236]])

documents = torch.tensor(
    [[ 0.8498,  1.4255, -1.3913, -0.0906,  2.6704, -1.5063, -1.5604, -0.4563,
        0.4762,  0.7897, -0.1102,  0.1176,  0.3902,  1.5095, -0.3534],
    [-0.9154, -0.1968,  0.5091,  0.0156, -1.6841, -1.1580,  1.2767,  2.6576,
        -0.3602,  0.4782,  0.7819,  0.7402, -0.8883, -0.1158,  1.0545],
    [-0.8661,  0.3513, -1.8400, -3.5891, -1.3286, -0.1409, -1.3466,  1.1086,
        0.4160,  2.5859,  0.0813, -0.5245,  0.1244,  0.3139,  1.2755],
    [ 0.1836, -1.2159, -0.5191, -1.5825,  0.4003, -0.6419, -1.1341,  0.2970,
        -1.1792,  2.1851,  2.3077,  0.3735,  1.4981,  0.6243,  1.2269],
    [-1.2163,  0.2481, -1.9740,  0.2509,  1.0521,  0.5903, -0.6070, -0.6650,
        -0.1618,  0.5526,  0.6654,  0.9530, -0.5084,  1.8372, -0.2625],
    [ 1.4968, -0.4212, -0.3566, -0.1982,  0.3722,  0.4442,  1.0164,  0.8380,
        -0.5248, -1.1686,  1.3973, -0.6910, -0.5832, -0.2636, -1.0497]])

labels = torch.eye(queries.size(0))
assert abs(criterion(queries, documents, labels).item() - 8.74177) < 1e-4

labels = torch.tensor(
    [[0.5000, 0.0000, 0.0000, 0.0000, 0.5000, 0.0000],
    [0.0000, 0.3333, 0.0000, 0.3333, 0.0000, 0.3333],
    [0.0000, 0.0000, 1.0000, 0.0000, 0.0000, 0.0000],
    [0.0000, 0.3333, 0.0000, 0.3333, 0.0000, 0.3333],
    [0.5000, 0.0000, 0.0000, 0.0000, 0.5000, 0.0000],
    [0.0000, 0.3333, 0.0000, 0.3333, 0.0000, 0.3333]])
assert abs(criterion(queries, documents, labels).item() - 7.48024) < 1e-4

## 3.Модель - 4 баллов

Реализуйте класс `Embedder`, который производит расчет эмбеддинга по выходу токенизатора. Вы вольны выбирать архитектуру, но суть работы модели должна сохраняться.

In [ ]:
class Embedder(nn.Module):
    def __init__(self, config):
        super().__init__()
# ---- Ваш код здесь ----
# ---- Конец кода ----

## 4.Тестовая метрика и инференс - 5 баллов

Напишите функцию `calc_recall(index, query_embeddings, query_ids, qrels, documents, columns, config)`, которая по тестовой выборке эмбеддингов запросов `query_embeddings`, описываемые id запросов `query_ids`, извлекает наиболее релевантные документы из индекса документов `index` (стройте по `sampled_documents`) и по парам позитивов `qrels` и отображению id индекса в id документов `documents` (см. класс `Columns` и определение датасета) расчитывает средний модифицированный `Recall@K` по всем запросам. `@K` берите несколько, задайте списком через `config`.

$$Recall@K=\frac{\#[\text{число релевантных запросу документов в top-K}]}{\min\left(\#[\text{число всех релевантных запросу документов}],\, \#[\text{документов в выдаче по запросу}]\right)}$$

Для построения индекса документов используйте `faiss`.

Напишите функцию `inference(embedder, texts, is_query, config)`, которая прогоняет эмбеддер по текстам, `is_query` -- флаг того, являются ли тексты запросами или нет (для задания `max_length` в токенизации).

Напишите функцию `test_retriever(embedder, test_queries, test_qrels, documents, columns, config)`, которая считает тестовую метрику (запускает `calc_recall`).

In [ ]:
# ---- Ваш код здесь ----
print("Считаем тестовую метрику")
# ---- Конец кода ----

In [ ]:
# ---- Ваш код здесь ----
print("Инференсим, тестируем")
# ---- Конец кода ----

## 5.Функция обучения - 10 баллов

Напишите функцию `train_retriever(embedder, train_queries, train_documents, train_qrels, test_queries, test_qrels, documents, columns, dataset_config, train_test_config, loss_config)`, которая готовит все loader'ы, обучает модель в контрастивном режиме и считает раз в эпоху тестовую метрику.

Поэкспериментируйте с функцией ошибки, сделайте линейную комбинацию из `loss(q, d)` и `loss(d, q)` для обучения. Посмотрите на влияние такой смены ролей запроса и документа на итоговые метрики.

Стоит использовать `gradient accumulation`, который несложно пишется руками.

In [ ]:
# ---- Ваш код здесь ----
print("Пишем обучение")
# ---- Конец кода ----

## 6.Сборка - 3 баллов

Соберите все вместе, обучите эмбеддер, посмотрите на метрики теста до обучения и после. Сделайте выводы.

Подсказка: да, может случиться так, что метрики не вырастут. Правильные выводы, почему так происходит, уберегут вас от потери баллов за этот пункт (в случае если не будет ошибок в реализации в предыдущих пунктах).

In [ ]:
# ---- Ваш код здесь ----
print("Определяем модель")
# ---- Конец кода ----

In [ ]:
# ---- Ваш код здесь ----
print("Тестируем модель до обучения")
# ---- Конец кода ----

In [ ]:
# ---- Ваш код здесь ----
print("Запускаем обучение")
# ---- Конец кода ----


In [ ]:
# ---- Ваш код здесь ----
print("Тестируем после")
# ---- Конец кода ----

Ваши выводы туть: **Бомбордиро Крокодило**

---